# Needed modules

In [1]:
%%time
# STEP 1
import sys
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import getpass

CPU times: user 48.8 ms, sys: 29.8 ms, total: 78.6 ms
Wall time: 187 ms


The second step is to initialize the urllib PoolManager and set the base URL for the API requests that will be sent to the GES DISC subsetting service.

In [2]:
# STEP 2
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

The third step defines a local general-purpose method that submits a JSON-formatted Web Services Protocol (WSP) request to the GES DISC server, checks for any errors, and then returns the response. This method is created for convenience as this task will be repeated more than once.

In [3]:
# STEP 3
# This method POSTs formatted JSON WSP requests to the GES DISC endpoint URL
# It is created for convenience since this task will be repeated more than once
def get_http_data(request):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty %s request' % response['methodname'])
        sys.exit(1)
    return response

In [26]:
# STEP 4
# Define the parameters for the data subset
product = 'M2I3NPASM_V5.12.4' #<=========== I have to change here
varNames =['T', 'RH', 'O3']   #<=========== I have to change here
minlon = -180                 #<=========== I have to change here
maxlon = 180                  #<=========== I have to change here
minlat = -90                  #<=========== I have to change here
maxlat = -45                  #<=========== I have to change here
begTime = '1980-01-01'        #<=========== I have to change here
endTime = '1980-01-05'        #<=========== I have to change here
begHour = '00:00'             #<=========== I have to change here
endHour = '00:00'             #<=========== I have to change here
#-----------------------------------------------------------------------------#
# Subset only the mandatory pressure levels (units are hPa)
# 1000 925 850 700 500 400 300 250 200 150 100 70 50 30 20 10 7 5 3 2 1 
dimName = 'lev'
dimVals = [1,4,7,13,17,19,21,22,23,24,25,26,27,29,30,31,32,33,35,36,37]
# Construct the list of dimension name:value pairs to specify the desired subset
dimSlice = []
for i in range(len(dimVals)):
    dimSlice.append({'dimensionId': dimName, 'dimensionValue': dimVals[i]})


``https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2I1NXASM.5.12.4/${yr}/${mn}/MERRA2_${id}.inst1_2d_asm_Nx.${yr}${mn}${dy}.nc4.nc4?QV2M,SLP,T2M,U10M,V10M,time,lat,lon``


In [21]:
# STEP 4
# Define the parameters for the data subset
b           #<=========== I have to change here
#-----------------------------------------------------------------------------#
# Subset only the mandatory pressure levels (units are hPa)
# 1000 925 850 700 500 400 300 250 200 150 100 70 50 30 20 10 7 5 3 2 1 
#dimName = 'lev'
#dimVals = [1,4,7,13,17,19,21,22,23,24,25,26,27,29,30,31,32,33,35,36,37]
# Construct the list of dimension name:value pairs to specify the desired subset
dimSlice = [0]
#for i in range(len(dimVals)):
#    dimSlice.append({'dimensionId': dimName, 'dimensionValue': dimVals[i]})

In [27]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True, 
        'data': [{'datasetId': product,
                  'variable' : varNames[0],
                  'slice': dimSlice
                 },
                 {'datasetId': product,
                  'variable' : varNames[1],
                  'slice'    : dimSlice
                 },
                 {'datasetId': product,
                  'variable' : varNames[2],
                  'slice': dimSlice
                 }]
    }
}

In [24]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True, 
        'data': [{'datasetId': product,
                  'variable' : varNames[0]
                 },
                 {'datasetId': product,
                  'variable' : varNames[1]
                 },
                 {'datasetId': product,
                  'variable' : varNames[2]
                 }]
    }
}

In [28]:
# STEP 6
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

Job ID: 6536a84103a4d7149a3bd92d
Job status: Accepted


At this point the job is running on the GES DISC server. The seventh step is to construct another JSON WSP status_request, with methodname parameter set to 'GetStatus'. The args parameter contains the extracted Job ID. The status_request is submitted periodically to monitor the job status as it changes from 'Accepted' to 'Running' to '100% completed'. When the job is finished check on the final status to ensure the job succeeded.

In [7]:
# STEP 7
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Job status: Succeeded (100% complete)
Job Finished:  Complete (M2I3NPASM_5.12.4)


After confirming that the job has finished successfully it is time to retrieve the results. The results of a subset request job are URLs: there are HTTP_Services URLs (one for every data granule in the time range of interest) plus links to any relevant documentation. Each HTTP_Services URL contains the specifics of the subset request encoded as facets. Data subsets and documentation files are downloaded using the requests Python library.

There are two ways to retrieve the list of URLs when the subset job is finished:

**Method 1:** Use the API method named GetResult. This method will return the URLs along with three additional attributes: a label, plus the beginning and ending time stamps for that particular data granule. The label serves as the filename for the downloaded subsets.

**Method 2:** Retrieve a plain-text list of URLs in a single shot using the saved JobID. This is a shortcut to retrieve just the list of URLs without any of the other metadata.

The next step for **Method 1** is to construct a third type of JSON WSP request that retrieves the results of this Job. When that request is submitted the results are returned in batches of 20 items, starting with item 0. The startIndex value in the results_request structure must be updated after each successive batch is retrieved.

In [8]:
# STEP 8 (Plan A - preferred)
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 6 out of 6 expected items


Below is the code for **Method 2**. Construct a request using the saved JobID and retrieve the results with the requests library. If the requests.get() method does not return an error, the URLs are stored locally and printed out for informational purposes.

In [9]:
# STEP 8 (Plan B)
# Retrieve a plain-text list of results in a single shot using the saved JobID

result = requests.get('https://disc.gsfc.nasa.gov/api/jobs/results/'+myJobId)
try:
    result.raise_for_status()
    urls = result.text.split('\n')
    for i in urls : print('\n%s' % i)
except :
    print('Request returned error code %d' % result.status_code)


https://goldsmr5.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I3NPASM.5.12.4/doc/MERRA2.README.pdf

https://goldsmr5.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2I3NPASM.5.12.4%2F1980%2F01%2FMERRA2_100.inst3_3d_asm_Np.19800101.nc4&SHORTNAME=M2I3NPASM&DATASET_VERSION=5.12.4&VARIABLES=T%2CRH%2CO3&VERSION=1.02&BBOX=-90%2C-180%2C-45%2C180&LABEL=MERRA2_100.inst3_3d_asm_Np.19800101.SUB.nc&SERVICE=L34RS_MERRA2&LAYERS=LAYER_1%2C4%2C7%2C13%2C17%2C19%2C21%2C22%2C23%2C24%2C25%2C26%2C27%2C29%2C30%2C31%2C32%2C33%2C35%2C36%2C37&FORMAT=nc4%2F

https://goldsmr5.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2I3NPASM.5.12.4%2F1980%2F01%2FMERRA2_100.inst3_3d_asm_Np.19800102.nc4&SHORTNAME=M2I3NPASM&DATASET_VERSION=5.12.4&VARIABLES=T%2CRH%2CO3&VERSION=1.02&BBOX=-90%2C-180%2C-45%2C180&LABEL=MERRA2_100.inst3_3d_asm_Np.19800102.SUB.nc&SERVICE=L34RS_MERRA2&LAYERS=LAYER_1%2C4%2C7%2C13%2C17%2C19%2C21%2C22%2C23%2C24%2C25%2C26%2C27%2C29%2C30%2

It is important to keep in mind that the results returned at this point are not data files but lists of URLs. Most of the URLs will contain HTTP_Services requests to actually do the subsetting and return the data, but some of them may be links to documentation files pertaining to the dataset in question.

It is worthwhile to separate the document URLs from the HTTP_services URLs in case the documentation has already been retrieved so they won't be downloaded again. The way we do this is to check for start and end attributes which are always associated with HTTP_services URLs. The remainder of the example code assumes the use of **Method 1** because it makes use of this extra metadata.

In [10]:
# Sort the results into documents and URLs

docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
# print('\nDocumentation:')
# for item in docs : print(item['label']+': '+item['link'])

The final step is to invoke each HTTP_Services URL and download the data files. The contents of the label attribute are used here as the output file name, but the name can be any string. It is important to download each file one at a time, in series rather than in parallel, to avoid overloading the GES DISC servers.

We show two methods for downloading the data files using either the requests.get() or the urllib.request() modules. Use one or the other, but not both. If the requests.get() method fails try the alternate code block, but be sure to update it with a proper Earthdata login name and password.

**Download with Requests Library:**  
In STEP 10, for the request.get() module to work properly, you must have a [HOME/.netrc](https://disc.gsfc.nasa.gov/data-access) file that contains the following text (configured with your own Earthdata userid and password): machine urs.earthdata.nasa.gov login [userid] password [password]. In the alternate STEP 10, you can provide your userid and password on-the-fly when running the code.

In [11]:
# STEP 10 
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn, "is downloaded")
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
        
print('Downloading is done and find the downloaded files in your current working directory')


HTTP_services output:
MERRA2_100.inst3_3d_asm_Np.19800101.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800102.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800103.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800104.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800105.SUB.nc is downloaded


**Alternative Download Method Using Native Python:**  
If the code above does not work in your local environment try this alternate method. Please enter your Earthdata userid and password when prompted while running the code.

In [12]:
# ATLERNATIVE STEP 10 
# Create a password manager to deal with the 401 response that is returned from
# Earthdata Login

# Create a password manager to deal with the 401 response that is returned from
# Earthdata Login

username = input("Provide your EarthData userid: ")
password = getpass.getpass("Provide your EarthData password: ")

password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)

# Create a cookie jar for storing cookies. This is used to store and return the session cookie #given to use by the data server
cookie_jar = CookieJar()
   
# Install all the handlers.
opener = urllib.request.build_opener (urllib.request.HTTPBasicAuthHandler (password_manager),urllib.request.HTTPCookieProcessor (cookie_jar))
urllib.request.install_opener(opener)
 
# Open a request for the data, and download files
print('\nHTTP_services output:')
for item in urls:
    URL = item['link'] 
    DataRequest = urllib.request.Request(URL)
    DataResponse = urllib.request.urlopen(DataRequest)

# Print out the result
    DataBody = DataResponse.read()

# Save file to working directory
    try:
        file_name = item['label']
        file_ = open(file_name, 'wb')
        file_.write(DataBody)
        file_.close()
        print (file_name, "is downloaded")
    except requests.exceptions.HTTPError as e:
         print(e)
            
print('Downloading is done and find the downloaded files in your current working directory')

Provide your EarthData userid: ojhoundegnonto2jpl
Provide your EarthData password: ········

HTTP_services output:
MERRA2_100.inst3_3d_asm_Np.19800101.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800102.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800103.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800104.SUB.nc is downloaded
MERRA2_100.inst3_3d_asm_Np.19800105.SUB.nc is downloaded


The two code blocks above replace Steps 4 and 5 in the complete workflow outlined in Example #1.

**Additional Info:**  
[How to Use the Web Services API for Subsetting](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting)  
[How to Use the Web Services API for Dataset Searching](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Dataset%20Searching)  
[Complete reference documentation for the GES DISC Subsetting Service API](https://disc.gsfc.nasa.gov/service/subset) 

<font size="1">THE SUBJECT FILE IS PROVIDED "AS IS" WITHOUT ANY WARRANTY OF ANY KIND, EITHER EXPRESSED, IMPLIED, OR STATUTORY, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTY THAT THE SUBJECT FILE WILL CONFORM TO SPECIFICATIONS, ANY IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR FREEDOM FROM INFRINGEMENT, ANY WARRANTY THAT THE SUBJECT FILE WILL BE ERROR FREE, OR ANY WARRANTY THAT DOCUMENTATION, IF PROVIDED, WILL CONFORM TO THE SUBJECT FILE. THIS AGREEMENT DOES NOT, IN ANY MANNER, CONSTITUTE AN ENDORSEMENT BY GOVERNMENT AGENCY OR ANY PRIOR RECIPIENT OF ANY RESULTS, RESULTING DESIGNS, HARDWARE, SOFTWARE PRODUCTS OR ANY OTHER APPLICATIONS RESULTING FROM USE OF THE SUBJECT FILE. FURTHER, GOVERNMENT AGENCY DISCLAIMS ALL WARRANTIES AND LIABILITIES REGARDING THIRD-PARTY SOFTWARE, IF PRESENT IN THE SUBJECT FILE, AND DISTRIBUTES IT "AS IS."

In [48]:
product = 'M2I1NXASM.5.12.4' #<=========== I have to change here
varName ='T10M'   #<=========== I have to change here
minlon = -160                 #<=========== I have to change here
maxlon = -140                  #<=========== I have to change here
minlat = 56                  #<=========== I have to change here
maxlat = 75                  #<=========== I have to change here
begTime = '2022-09-01'        #<=========== I have to change here
endTime = '2022-09-05'        #<=========== I have to change here
#begHour = '00:00'             #<=========== I have to change here
#endHour = '00:00'  

# STEP 4
# Define the parameters for the second subset example
product = 'M2T1NXSLV_5.12.4'
varNames =['TQV', 'TQL', 'TQI']
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
begTime = '1980-01-04'
endTime = '1980-01-05'


diurnalAggregation = '1'
interp = 'remapbil'
destGrid = 'cfsr0.5a'

In [47]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True,
        'diurnalAggregation': diurnalAggregation,
        'mapping': interp,
        'grid'  : destGrid,
        'data': [{'datasetId': product,
                  'variable' : varNames[0]
                 }],
                 {'datasetId': product,
                  'variable' : varNames[1]
                 },
                 {'datasetId': product,
                  'variable' : varNames[2]
                 }]
    }
}

In [72]:
#product = 'M2T1NXSLV_5.12.4'
#varNames =['TQV', 'TQL', 'TQI']
#varName ='TQV'

product = 'M2I1NXASM.5.12.4' #<=========== I have to change here
#product = 'M2IMNXASM.5.12.4'
varName =['T2M','T10M', 'TO3']

minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
begTime = '2020-01-04'
endTime = '2020-01-05'

In [75]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True,        
        'data': [{'datasetId': product,
                  'variable' : varNames[0]
                 },
                  {'datasetId': product,
                  'variable' : varNames[1]
                 },
                 {'datasetId': product,
                  'variable' : varNames[2]
                 }]
    }
}

In [74]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True,        
        'data': [{'datasetId': product,
                  'variable' : varName
                 }]
    }
}

In [76]:
# STEP 6
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

KeyError: 'methodname'

At this point the job is running on the GES DISC server. The seventh step is to construct another JSON WSP status_request, with methodname parameter set to 'GetStatus'. The args parameter contains the extracted Job ID. The status_request is submitted periodically to monitor the job status as it changes from 'Accepted' to 'Running' to '100% completed'. When the job is finished check on the final status to ensure the job succeeded.

In [7]:
# STEP 7
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Job status: Succeeded (100% complete)
Job Finished:  Complete (M2I3NPASM_5.12.4)


After confirming that the job has finished successfully it is time to retrieve the results. The results of a subset request job are URLs: there are HTTP_Services URLs (one for every data granule in the time range of interest) plus links to any relevant documentation. Each HTTP_Services URL contains the specifics of the subset request encoded as facets. Data subsets and documentation files are downloaded using the requests Python library.

There are two ways to retrieve the list of URLs when the subset job is finished:

**Method 1:** Use the API method named GetResult. This method will return the URLs along with three additional attributes: a label, plus the beginning and ending time stamps for that particular data granule. The label serves as the filename for the downloaded subsets.

**Method 2:** Retrieve a plain-text list of URLs in a single shot using the saved JobID. This is a shortcut to retrieve just the list of URLs without any of the other metadata.

The next step for **Method 1** is to construct a third type of JSON WSP request that retrieves the results of this Job. When that request is submitted the results are returned in batches of 20 items, starting with item 0. The startIndex value in the results_request structure must be updated after each successive batch is retrieved.

In [36]:
# STEP 8 (Plan A - preferred)
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 3 out of 3 expected items


It is important to keep in mind that the results returned at this point are not data files but lists of URLs. Most of the URLs will contain HTTP_Services requests to actually do the subsetting and return the data, but some of them may be links to documentation files pertaining to the dataset in question.

It is worthwhile to separate the document URLs from the HTTP_services URLs in case the documentation has already been retrieved so they won't be downloaded again. The way we do this is to check for start and end attributes which are always associated with HTTP_services URLs. The remainder of the example code assumes the use of **Method 1** because it makes use of this extra metadata.

In [38]:
# Sort the results into documents and URLs

docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
# print('\nDocumentation:')
# for item in docs : print(item['label']+': '+item['link'])

The final step is to invoke each HTTP_Services URL and download the data files. The contents of the label attribute are used here as the output file name, but the name can be any string. It is important to download each file one at a time, in series rather than in parallel, to avoid overloading the GES DISC servers.

We show two methods for downloading the data files using either the requests.get() or the urllib.request() modules. Use one or the other, but not both. If the requests.get() method fails try the alternate code block, but be sure to update it with a proper Earthdata login name and password.

**Download with Requests Library:**  
In STEP 10, for the request.get() module to work properly, you must have a [HOME/.netrc](https://disc.gsfc.nasa.gov/data-access) file that contains the following text (configured with your own Earthdata userid and password): machine urs.earthdata.nasa.gov login [userid] password [password]. In the alternate STEP 10, you can provide your userid and password on-the-fly when running the code.

In [39]:
# STEP 10 
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn, "is downloaded")
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
        
print('Downloading is done and find the downloaded files in your current working directory')


HTTP_services output:
MERRA2_100.tavg1_2d_slv_Nx.19800104.SUB.nc is downloaded
MERRA2_100.tavg1_2d_slv_Nx.19800105.SUB.nc is downloaded
